In [3]:
!pip install beautifulsoup4

     |████████████████████████████████| 122kB 7.2MB/s eta 0:00:01


In [11]:
!pip3 install lxml
!pip install lxml

     |████████████████████████████████| 5.5MB 5.9MB/s eta 0:00:01     |███████                         | 1.2MB 5.1MB/s eta 0:00:01     |██████████▉                     | 1.9MB 5.1MB/s eta 0:00:01


In [5]:
!pip install geocoder

     |████████████████████████████████| 102kB 3.1MB/s ta 0:00:01


In [6]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py

In [145]:
#Importing necessary libraries
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import random
import lxml
from IPython.display import display_html
import pandas as pd
import numpy as np
from IPython.display import Image 
from IPython.core.display import HTML 
from lxml import etree
import folium # map rendering library
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source, 'lxml')
print(soup.title)

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned': #Pass the rows which have a Not Assigned value
        pass
    else:
        cell['PostalCode'] = row.p.text[:3] # first three symbols are Postalcode
        cell['Borough'] = (row.span.text).split('(')[0] 
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
df=pd.DataFrame(table_contents)
df

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df


<title>List of postal codes of Canada: M - Wikipedia</title>


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
df.shape 

(103, 3)

In [5]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

--2021-04-04 09:36:05--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 52.116.127.226, 52.116.127.228
Connecting to cocl.us (cocl.us)|52.116.127.226|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-04-04 09:36:06--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-04-04 09:36:06--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [followin

In [6]:
longlat= pd.read_csv("Geospatial_Coordinates.csv", delimiter=",")
longlat

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [7]:
df_ll= df.join(longlat.set_index('Postal Code'), on='PostalCode')
df_ll

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


#### Lets find Toronto coordinats for the map

In [8]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create map using coordinates and adding Boroughs and Neighborhoods

In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_ll['Latitude'], df_ll['Longitude'], df_ll['Borough'], df_ll['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Separating Boroughs which contain word Toronto, from the dataset

In [49]:
toronto_data = df_ll[df_ll['Borough'].str.contains('Toronto',regex=False)].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


#### Creating map using only Boroughs which 

In [50]:
# create map of Toronto using latitude and longitude values
map_toronto1 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto1)  
    
map_toronto1

## Foursquare api credentials

In [51]:
CLIENT_ID = 'ULJTMIHAXZ2UWTNCYSCR2MOU2UCKLCHTV1IXWJPOCISAVPXF' # your Foursquare ID
CLIENT_SECRET = 'RB3ULC4KCTMLNVUUXWGLSOQRYOXPFSNMEPQIJIKL33PQKE3D' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ULJTMIHAXZ2UWTNCYSCR2MOU2UCKLCHTV1IXWJPOCISAVPXF
CLIENT_SECRET:RB3ULC4KCTMLNVUUXWGLSOQRYOXPFSNMEPQIJIKL33PQKE3D


#### Get first neighborhuud from the Toronto data and its coordinates

In [58]:
toronto_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [60]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


#### Now, let's get the top 20 venues that are in Regent Park, Harbourfront within a radius of 200 meters.

In [65]:
LIMIT = 20
radius = 200
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=ULJTMIHAXZ2UWTNCYSCR2MOU2UCKLCHTV1IXWJPOCISAVPXF&client_secret=RB3ULC4KCTMLNVUUXWGLSOQRYOXPFSNMEPQIJIKL33PQKE3D&v=20180605&ll=43.6542599,-79.3606359&radius=200&limit=20'

In [66]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60699a77855bb878e6f8bc5c'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.6560599018, 'lng': -79.35815270005945},
   'sw': {'lat': 43.6524598982, 'lng': -79.36311909994056}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65355870959944,
          'lng': -79.36180945913513}],
        'distance': 122,
       

#### Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [70]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Clean the json file

In [71]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Body Blitz Spa East,Spa,43.654735,-79.359874
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,The Extension Room,Gym / Fitness Center,43.653313,-79.359725


#### Define a function to pull the venues data for all tge neighborhoods from Foursquare and categorize it

In [76]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Let's use the function created above to gather all the neighborhoods venues.

In [77]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

#### Let's check the size of the resulting dataframe

In [78]:
print(toronto_venues.shape)
toronto_venues.head()

(625, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


How many venues returned to each neighborhood

In [80]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"Brockton, Parkdale Village, Exhibition Place",20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,20,20,20,20,20,20
Christie,16,16,16,16,16,16
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
Davisville,20,20,20,20,20,20
Davisville North,8,8,8,8,8,8


How many unique venue categories there are

In [82]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 173 unique categories.


Analyze each neighborhood

In [132]:
# one hot encoding and dummify the venue categories
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

toronto_onehot.head()

,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
toronto_onehot.shape

(625, 173)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [133]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.00,...,0.05,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.125,0.1875,0.125,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,Central Bay Street,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.05,...,0.00,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0
4,Christie,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.00,...,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


New dataset size

In [134]:
toronto_grouped.shape

(39, 173)

Each neighborhood top 3 venues


In [135]:
num_top_venues = 3

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.10
1      Farmers Market  0.10
2   French Restaurant  0.05


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.15
1     Coffee Shop  0.10
2  Breakfast Spot  0.10


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12


----Central Bay Street----
         venue  freq
0  Coffee Shop  0.20
1   Poke Place  0.05
2    Gastropub  0.05


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12


----Church and Wellesley----
         venue  freq
0  Coffee Shop  0.05
1    Juice Bar  0.05
2  Pizza Place  0.05


----Commerce Court, Victoria Hotel----
         venue  freq
0         Café   0.2
1  Coffee Shop   0.1
2    Gastropub   0.1


----Davisville----
           

#### Let's put that into a _pandas_ dataframe

First, let's write a function to sort the venues in descending order.

In [136]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 5 venues for each neighborhood.

In [139]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Farmers Market,Seafood Restaurant,Museum,Jazz Club,Breakfast Spot
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Stadium,Nightclub
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Plane
3,Central Bay Street,Coffee Shop,Park,Italian Restaurant,Ramen Restaurant,Poke Place
4,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant


### Cluster neighborhoods using kmeans

In [140]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 2, 1, 1, 2, 1, 1, 2, 1], dtype=int32)

Add cluster labels to sorted venues

In [141]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

Merging table with Toronto data to get latidudes and longitudes

In [143]:
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Breakfast Spot,Chocolate Shop,Historic Site
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Café,Clothing Store,Sporting Goods Shop,Mexican Restaurant,Comic Shop
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Beer Bar,Food Truck,Japanese Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Asian Restaurant,Trail,Health Food Store,Pub,Creperie
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Farmers Market,Seafood Restaurant,Museum,Jazz Club,Breakfast Spot


Lets put the clustered Neighborhoods on the map

In [147]:
# create map of Toronto using latitude and longitude values
map_toronto_clust = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto_clust)
    
map_toronto_clust

Lets see which venues and where each cluster contains

In [157]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,The Danforth East,0,Park,Intersection,Convenience Store,Yoga Studio,Dance Studio
33,Rosedale,0,Park,Playground,Trail,Cosmetics Shop,Discount Store


In [156]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Regent Park, Harbourfront",1,Coffee Shop,Park,Breakfast Spot,Chocolate Shop,Historic Site
1,"Garden District, Ryerson",1,Café,Clothing Store,Sporting Goods Shop,Mexican Restaurant,Comic Shop
2,St. James Town,1,Coffee Shop,Café,Beer Bar,Food Truck,Japanese Restaurant
5,Central Bay Street,1,Coffee Shop,Park,Italian Restaurant,Ramen Restaurant,Poke Place
6,Christie,1,Grocery Store,Café,Park,Candy Store,Italian Restaurant
7,"Richmond, Adelaide, King",1,Café,Coffee Shop,Asian Restaurant,Restaurant,Seafood Restaurant
8,"Dufferin, Dovercourt Village",1,Bakery,Pharmacy,Smoke Shop,Bank,Supermarket
13,"Toronto Dominion Centre, Design Exchange",1,Coffee Shop,Café,Japanese Restaurant,Gym / Fitness Center,Pub
14,"Brockton, Parkdale Village, Exhibition Place",1,Café,Coffee Shop,Breakfast Spot,Stadium,Nightclub
16,"Commerce Court, Victoria Hotel",1,Café,Japanese Restaurant,Gastropub,Coffee Shop,Steakhouse


In [155]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,The Beaches,2,Asian Restaurant,Trail,Health Food Store,Pub,Creperie
4,Berczy Park,2,Farmers Market,Seafood Restaurant,Museum,Jazz Club,Breakfast Spot
10,"Harbourfront East, Union Station, Toronto Islands",2,Park,Plaza,Salad Place,Italian Restaurant,Deli / Bodega
11,"Little Portugal, Trinity",2,Bar,Yoga Studio,Asian Restaurant,Pizza Place,Cuban Restaurant
12,"The Danforth West, Riverdale",2,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pizza Place
15,"India Bazaar, The Beaches West",2,Park,Fast Food Restaurant,Steakhouse,Pet Store,Pizza Place
18,Lawrence Park,2,Park,Business Service,Bus Line,Swim School,Creperie
19,Roselawn,2,Home Service,Garden,Music Venue,Creperie,Distribution Center
20,Davisville North,2,Park,Department Store,Playground,Sandwich Place,Gym / Fitness Center
21,Forest Hill North & West,2,Jewelry Store,Sushi Restaurant,Trail,Mexican Restaurant,Dog Run


In [154]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
29,"Moore Park, Summerhill East",3,Trail,Restaurant,Yoga Studio,Cuban Restaurant,Distribution Center
